In [1]:
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
from copy import deepcopy

1) Реализовать алгоритм слияния n (n=2**k) отсортированных списков (произвольной длины) в один отсортированный список.

In [2]:
def merge_lists(lists):
    min_values = []
    merged_list = []
    lists = list(deepcopy(lists))
    
    for l in lists:
        min_values.append(l.pop(0))

    while len(lists) != 0:
        min_val = min(min_values)
        min_val_idx = min_values.index(min_val)
        merged_list.append(min_values.pop(min_val_idx))

        if len(lists[min_val_idx]) == 0:
            lists.pop(min_val_idx)
        else:
            val_to_insert = lists[min_val_idx].pop(0)
            min_values.insert(min_val_idx, val_to_insert)

    return merged_list

In [3]:
a = [2, 4, 11]
b = [6, 8, 10]

In [4]:
merge_lists([a, b])

[2, 4, 6, 8, 10, 11]

2) С помощью Dask реализовать генерацию n отсортированных списков целых значений (для каждого списка задается собственная длина) и сохранение каждого списка в отдельный файл (в файле хранятся 3 столбца: номерспска, номерпо_порядку, значение).

In [5]:
def get_sorted_array(length):
    a = np.random.randint(100000, size=length)
    return np.cumsum(a)

In [6]:
get_sorted_array(10)

array([ 64755, 100554, 197043, 251897, 292356, 298413, 315186, 337719,
       437185, 459672])

In [7]:
@dask.delayed
def get_random_array(size):
    return np.random.randint(100000, size=size)

@dask.delayed
def add(x, y):
    return x+y

def get_sorted_array(size):
    a = get_random_array(size)
    output = []
    output.append(a[0])
    for i in range(1, size):
        output.append(add(output[i-1], a[i]))
    return output


def get_n_sorted_arrays(array_sizes):
    output = []
    for size in array_sizes:
        output.append(get_sorted_array(size))
    return output

In [8]:
out = get_n_sorted_arrays([3,4,5])
result = dask.compute(*out)

In [9]:
result

([90116, 153459, 220982],
 [74668, 121438, 131297, 148755],
 [81478, 152511, 251706, 251787, 281870])

In [77]:
@dask.delayed
def transform_lists(lst, lst_id):
    out = []
    for i in range(len(lst)):
        out.append([lst_id, i, lst[i]])
    return out

table = []
for lst_id in range(0, len(result)):
    rows = transform_lists(result[lst_id], lst_id)
    table = table + rows

In [78]:
table.compute()

[[0, 0, 35518],
 [0, 1, 70055],
 [0, 2, 134325],
 [1, 0, 58651],
 [1, 1, 124895],
 [1, 2, 132549],
 [1, 3, 130046],
 [2, 0, 58539],
 [2, 1, 142731],
 [2, 2, 125848],
 [2, 3, 128686],
 [2, 4, 142594]]

In [87]:
headers = ['list_id', 'element_id', 'element_value']
dframe = pd.DataFrame(table.compute(), columns=headers)
dframe.to_csv('sol_2.csv')

3) Применить решение задачи 1 для сортировки данных, сгенерированных в задаче 2

In [10]:
result

([90116, 153459, 220982],
 [74668, 121438, 131297, 148755],
 [81478, 152511, 251706, 251787, 281870])

In [11]:
merge_lists(result)

[74668,
 81478,
 90116,
 121438,
 131297,
 148755,
 152511,
 153459,
 220982,
 251706,
 251787,
 281870]

4) При помощи dask.delayed распараллелить решение задачи 1 и применить к данным, сгенерированным в задаче 2

5) Модифицировать 4 - обрабатывать данные тем же алгоритмом с помощью pandas.DataFrame

6) C помощью разработанного алгоритма и возможностей Dask.DataFrame отсортировать данные, которые обрабатывались на сминаере по Dask.DataFrame (столбцы: ,id,names,amount,new) по столбцу amount.